In [ ]:
from google.colab import drive
drive.mount('/content/drive')



## ----------------------- please set up data path echoDynamic.zip --------------------------------------------
## --------------------------- paths where models are going to be saved ---------------------------------------




Mounted at /content/drive


In [ ]:
import os,zipfile,keras
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Major_Project/EchoNet-Dynamic.zip', 'r')
zip_ref.extractall('/content/tmp')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np 
import cv2
import keras
from keras.layers import Conv2D,Input,LeakyReLU,BatchNormalization,Dropout,Concatenate,Dropout,GlobalAveragePooling2D,Dense,Flatten
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import random 
PATH = {
    'data_path' : "/content/tmp/EchoNet-Dynamic/", 
    'save_model': '/content/drive/MyDrive/Major Project/',
    'figure_path' : '/content/drive/MyDrive/Major_Project/Figure/'
}


# ------------------------------------------------------- global DATA --------------------------
# ------------------------------------ this will cause video data to erase --------------------
# -------------------------------------- so think before execution ---------------------------

TRAIN_SAMPLE_LOW = 0
TRAIN_SAMPLE_HIGH = 9000 ## make it 6K 
TEST_SAMPLE_LOW = 7000 ## 6K
TEST_SAMPLE_HIGH = 10000 ## 10K 
COMPRESSOR_EPOCHS = 5
PREDICTOR_EPOCHS = 10

fileList = pd.read_csv(PATH['data_path'] + 'FileList.csv')
inputSize = TRAIN_SAMPLE_HIGH - TRAIN_SAMPLE_LOW + 1
videoArrays = []
EF = []

In [ ]:
## -------------------------------------------------------------- preprocessing -----------------------------------------------------------

def VideoPreprocess(PATH,lower,upper):
  ## EF and videoArrays generated
  for i in range(lower,upper): 
    dfRow = fileList.iloc[i]
    fileName = dfRow.FileName
    path = PATH['data_path'] + "/Videos/" + fileName + '.avi'
    cap = cv2.VideoCapture(path)
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if(frameCount < 100):
       continue
    buf = np.empty((112, 112,frameCount), np.dtype('uint8')) ### creating 50 frames size buffer
    frameId = 0
    for frameId in range(frameCount): 
      ret,frame = cap.read()
      grayScale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      buf[:,:,frameId] = grayScale
      del frame 
    videoArrays.append(buf)
    EF.append(dfRow.EF)
  return 

# --------------------------------------------- Layers ------------------------------------------


def convS(input,filters,batch,stride,drop = 0.0): 
  ip = Input(shape = input.shape[1:])
  output = Conv2D(filters = filters,kernel_size = (3,3),strides=stride)(ip)
  if batch: 
    output = BatchNormalization()(output)
  output = LeakyReLU(alpha=.2)(output)
  if drop: 
    output = Dropout(drop)(output)
  model = Model(ip,output)
  return model(input)
def concatenatedLayers(input1,input2,filters,batch,drop):
  ip1 = Input(shape = input1.shape[1:])
  ip2 = Input(shape = input2.shape[1:])
  concat = Concatenate()([ip1,ip1])
  output = Conv2D(filters = filters,kernel_size = (3,3),strides=(1,1),padding='same')(concat)
  if batch: 
    output = BatchNormalization()(output)
  output = LeakyReLU(alpha=.2)(output)
  if drop:
    output = Dropout(drop)(output)
  model = Model([ip1,ip2],output)
  return model([input1,input2])
def convLayer(input,filters,batch,drop): 
  ip = Input(shape = input.shape[1:])
  output = Conv2D(filters = filters,kernel_size = (3,3),strides=(1,1),padding='same')(ip)
  if batch: 
    output = BatchNormalization()(output)
  output = LeakyReLU(alpha=.2)(output)
  if drop:
    output = Dropout(drop)(output)
  model = Model(ip,output)
  return model(input)


# ---------------------------------------------------------- generator MODEL ---------------------------------------------------------------------

def generation(): 
  input = Input(shape=(112,112,50)) 
  gen = convLayer(input = input,filters = 64,batch = True,drop = 0.0)
  gen = convLayer(input = gen,filters = 128,batch = True,drop = 0.2)
  gen = convLayer(input = gen,filters = 256,batch = True,drop = .4)
  gen = concatenatedLayers(input1 = gen,input2 = input,filters = 64,batch = False, drop = 0.2) 
  gen = convLayer(input = gen,filters = 32,batch = False,drop = .1)
  gen = convLayer(input = gen,filters = 16,batch = False,drop = 0.0)
  gen = Conv2D(filters = 1,kernel_size = (3,3),strides=(1,1),padding='same',activation = 'tanh')(gen)
  model = Model(input,gen) 
  return model

# ----------------------------------------------------------- regenerator model ----------------------------------------------------------------------------

def regeneration():
  inputReal = Input(shape = (112,112,2))
  regen = convLayer(input = inputReal,filters = 16,batch = True,drop = 0.0)
  regen = convLayer(input = regen,filters = 32,batch = True,drop = 0.0)
  regen = convLayer(input = regen,filters = 64,batch = True,drop = 0.1)
  regen = concatenatedLayers(input1 = regen,input2 = inputReal,filters = 128,batch = True, drop = 0.2)
  regen = convLayer(input = regen,filters = 128,batch = False,drop = 0.1)
  regen = convLayer(input = regen,filters = 64,batch = False,drop = 0.0)
  regen = Conv2D(filters = 50,kernel_size = (3,3),strides = (1,1), padding = "same",activation = 'tanh')(regen)
  model = Model(inputReal,regen)
  return model

# ----------------------------------------------------------- compressor MODEL 

def compressor(PATH,lr = 0.001):
  image = Input(shape = (112,112,1))
  volume = Input(shape = (112,112,50))
  generate = generation() 
  regenerate = regeneration()
  generatedImage = generate(volume)
  images = Concatenate()([image,generatedImage])
  finalVolume = regenerate(images)
  regen = Model([image,volume],finalVolume)
  regen.compile(optimizer = Adam(learning_rate = lr),loss = 'mae')
  plot_model(generate, to_file=PATH['figure_path'] + 'generator.png', show_shapes=True, show_layer_names=False)
  plot_model(regenerate, to_file=PATH['figure_path'] + 'regenerator.png', show_shapes=True, show_layer_names=False)
  plot_model(regen, to_file=PATH['figure_path'] + 'pipeline.png', show_shapes=True, show_layer_names=False)
  return regen,generate


# ---------------------------------------------------------------- compressor generator ------------------------------------------------------
class compressorDataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  def __init__(self, PATH):
    'Initialization'
    self.batch_size = 1
    self.len = len(videoArrays) ## global fileList 
    self.on_epoch_end()  
    self.index = 0

  def __len__(self):
    'Denotes the number of batches per epoch'
    return self.len 

  def __getitem__(self, index):
    'Generate one batch of data'
    X, y = self.__data_generation(self.index)
    return X, y

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    self.index = random.randrange(0,self.len-1) ## taking random number in range 

  def __data_generation(self, index):
    'Generates data containing batch_size samples'

    ## it's in 112x112xFRAMES shape
    video = videoArrays[index]
    video = (video - 127.5)/127.5 ## tranforming Video Arrays 
    datasetX = np.empty((video.shape[0]//50,112,112,50))
    datasetY = np.empty((video.shape[0]//50,112,112,1))
    datasetZ = np.empty((video.shape[0]//50,112,112,50))
    i = 0
    while(i + 50 <= video.shape[0]): 
      x = video[:,:,i:i+50]
      image = x[:,:,1:2]
      datasetX[i//50] = x
      datasetZ[i//50] = x 
      datasetY[i//50] = image
      i+=50
      # del image,x 
    return [datasetY,datasetX],datasetZ


In [ ]:
## -------------------------------------------- video.npy generation ------------------------------------------

VideoPreprocess(PATH,TRAIN_SAMPLE_LOW,TRAIN_SAMPLE_HIGH) 

## 23 GB for 9000 dataset

In [ ]:
# ------------------------- training compressor ------------------------------


final,gen = compressor(PATH,lr = .0005)
generator = compressorDataGenerator(PATH)
final.fit_generator(generator,epochs = COMPRESSOR_EPOCHS,workers = 4)
final.save(PATH['save_model'] + 'wholeTestingFinal.h5')
gen.save(PATH['save_model'] + 'wholeTestingGen.h5')


<ipython-input-6-164d4554d703>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final.fit_generator(generator,epochs = COMPRESSOR_EPOCHS,workers = 4)


Epoch 1/5
8705/8705 [==============================] - 504s 56ms/step - loss: 0.0464
Epoch 2/5
8705/8705 [==============================] - 494s 57ms/step - loss: 0.0452
Epoch 3/5
8705/8705 [==============================] - 493s 57ms/step - loss: 0.0324
Epoch 4/5
8705/8705 [==============================] - 493s 57ms/step - loss: 0.0394
Epoch 5/5
8705/8705 [==============================] - 494s 57ms/step - loss: 0.0283


In [ ]:
del videoArrays,EF
videoArrays = []
EF = []
VideoPreprocess(PATH,9000,10000) 

In [ ]:
generator = compressorDataGenerator(PATH)
final.fit_generator(generator,epochs = COMPRESSOR_EPOCHS,workers = 4)
final.save(PATH['save_model'] + 'wholeTestingFinalWhole.h5')
gen.save(PATH['save_model'] + 'wholeTestingGenWhole.h5')

Epoch 1/5


<ipython-input-10-08515563f233>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final.fit_generator(generator,epochs = COMPRESSOR_EPOCHS,workers = 4)


961/961 [==============================] - 55s 57ms/step - loss: 0.0467
Epoch 2/5
961/961 [==============================] - 54s 57ms/step - loss: 0.0415
Epoch 3/5
961/961 [==============================] - 55s 57ms/step - loss: 0.0437
Epoch 4/5
961/961 [==============================] - 54s 57ms/step - loss: 0.0501
Epoch 5/5
961/961 [==============================] - 55s 57ms/step - loss: 0.0331


In [ ]:
del videoArrays,EF
videoArrays = []
EF = []
VideoPreprocess(PATH,0,1000) 
generator = compressorDataGenerator(PATH)
final.fit_generator(generator,epochs = 2,workers = 4)

Epoch 1/2
  1/969 [..............................] - ETA: 1:59 - loss: 0.0898

<ipython-input-11-0b495427fbe8>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final.fit_generator(generator,epochs = 2,workers = 4)


969/969 [==============================] - 55s 57ms/step - loss: 0.0486
Epoch 2/2
969/969 [==============================] - 55s 57ms/step - loss: 0.0485


In [ ]:
del videoArrays,EF
videoArrays = []
EF = []
VideoPreprocess(PATH,5000,6000) 
generator = compressorDataGenerator(PATH)
final.fit_generator(generator,epochs = 1,workers = 4)

  1/961 [..............................] - ETA: 1:51 - loss: 0.1010

<ipython-input-12-345609a833aa>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final.fit_generator(generator,epochs = 1,workers = 4)


961/961 [==============================] - 55s 58ms/step - loss: 0.0389


In [ ]:
final.save(PATH['save_model'] + 'wholeTestingFinalGeneralized.h5')
gen.save(PATH['save_model'] + 'wholeTestingGeneralized.h5')